#车牌识别faster rcnn

#RCNN，借鉴了卷积网络CNN在分类上的优点，通过region proposal（具体是Selective Search）将图像划分为若干局部区域，然后把这些区域分别输入到CNN中，提取区域的特征feature，加上分类器，判断feature对应的区域是属于具体某类object还是背景

#Fast RCNN
RCNN存在着重复计算的问题，使用SelectiveSearch得到的region有几千个，多数都是互相重叠，重叠部分会被多次重复提取feature，于是Fast-RCNN应运而生，跟RCNN最大区别就是Fast-RCNN只卷积处理一次，这样一张图片只需要提取一次feature，大大提高了速度

#Faster RCNN
Fast-RCNN的速度瓶颈在Region proposal上。将SelectiveSearch(region proposal，用vgg16代替)的一张图片划分成nxn个区域，每个区域给出9个不同ratio和scale的proposal，输出的是对输入的固定proposal是属于背景还是前景的判断，第一个Region proposal nework（vgg16）的输出再输入第二个Fast-RCNN做更精细的分类和Boundingbox的位置修正，第二个网络比较简单。

如上所述，有两层网络，第一层是vgg16，用来找到标注的区域，第二层网络，进行精细分类


#有两种模式，
第一种，直接标注号码区域，使用区域标注功能进行号码识别。因为第二层网络比较简单，不能识别复杂对象，而且因为显存的原因，输入图像尺寸受限，导致当中的对象尺寸更小，导致识别率提不上去。

#第二种，分成两个过程，
#第一部分利用fasterrcnn的region proposal功能，把车牌区域找出来。这个步骤的输入训练图像为原图的缩小。
#第二部分，首先使用第一步的网络，输入若干原始图像得到车牌区域，切割出来，调整为尺寸一致的车牌图像文件。这里的车牌图像文件，相比模式1中缩小后的图像中的车牌部分尺寸，不可同日而语。
然后标注这些车牌图像文件中的字符区域和目标。将之作为训练数据进行第二次训练。



选择faster-rcnn的pytorch版
数据集按照版本中已经实现的voc2007数据集格式整理
有这个几个步骤：
1. 数据集目录为$faster-rcnn-root/data/VOCdevkit2007/VOC2007
  $dataset/Annotations为标注目录，标注要求存在object项
  $dataset/JPEGImages为图像目录
  $dataset/ImageSets/Main为数据集定义目录，至少定义train.txt，内容为JPEGImages中图像文件的名称，不带后缀
  如果数据集有更新，需要删除$faster-rcnn-root/data/chache/voc_2007_$datasetname_gt_roidb.pkl
2.修改配置文件$faster-rcnn-root/experiments/cfgs/faster_rcnn_end2end.yml
  TRAIN下的SCALES和MAXSIZE，分别是输入图像的高度和宽度
3.修改代码
  $faster-rcnn-root/train.py，imdb_name改成'voc_2007'加上上文定义的数据集名字，缺省是trainval.txt
        迭代次数end_step改成自己需要的
  $faster-rcnn-root/faster_rcnn/faster_rcnn.py，
  a. class FasterRCNN(nn.module)，中将SCALES和MAXSIZE改为和配置文件中一致
  b. 对象种类数n_classes和对象名称定义classes改为和实际一致。这里‘__background__'必须有且为第一个位置。
  c. PIXEL_MEANS改为和实际一致，有利于提高精度

第一种模式，训练数据集500，迭代12万次，验证精度50%，48张数据集，完全正确的数量。
第二种模式，第一部分，车牌切割，200张数据集，车牌切割准确度80%，50张图片完全切割正确的数量（不缺少字符）